In [1]:
# Private Preview edition
# ! pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004
! pip show azure-search-documents

Name: azure-search-documents
Version: 11.4.0a20230509004
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: C:\Users\shchitt\AppData\Local\anaconda3\envs\aoai\Lib\site-packages
Requires: azure-common, azure-core, isodate
Required-by: 


In [2]:
# loading the API keys in environment variables
import os
import os, json
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [5]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['CHAT_MODEL_NAME']
embedding_model = os.environ['EMBEDDING_MODEL_NAME']

In [7]:
vector_store_endpoint: str = os.environ['AZURE_COGNITIVE_SEARCH_ENDPOINT']
vector_store_password: str = os.environ['AZURE_COGNITIVE_SEARCH_KEY']
index_name: str = "langchain-rag-demo"

In [8]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=embedding_model, chunk_size=1)
type(embeddings)

langchain.embeddings.openai.OpenAIEmbeddings

In [9]:
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_endpoint,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [10]:
from langchain.document_loaders import UnstructuredFileLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# loader = UnstructuredFileLoader("files/state_of_the_union.txt")
loader = TextLoader("files/state_of_the_union.txt", encoding="utf-8")
documents = loader.load()
len(documents)

1

In [11]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

42

In [12]:
vector_store.add_documents(documents=docs)

['Nzk5MWE4NGUtYTQzMy00ZmY5LThlZTgtOGQ2Y2Y1Zjk1ZWFj',
 'MDhjM2ExNjUtN2M3Yy00YzU0LWE2ZWUtYTVhNDM3NTg3M2Y2',
 'ODI2YzFhOTktYjI5Yi00YTM4LTk0ZjQtY2QyY2E2MjgyZjZj',
 'MDk4MGYwZjktNWQ1MC00YzJlLWI2ZTYtMTEwNjBjNjE5MzUy',
 'NGZmNjU0NTUtYjQwZS00YzZlLThiOTMtN2I1OTA0NTA0ZGNk',
 'YmQ3MDBhMzgtMjk2Yi00ZjAzLTlkMjItYjAzYTBjMWIyNzIx',
 'MTM5ZTQwMDItMTEyZC00MTRkLWJmYmItM2VjYjg5NWI2NmY1',
 'ZmYxM2FkNjgtYzhhMi00YTMyLWE3ZTQtODc2ZTZhNGExYjc0',
 'YjY2OTY1NzAtNzMzNC00NmJiLTkyNzEtNDhmM2Y0OGE4NTA2',
 'YWM5YWYzYzMtYTY1NS00YTcyLWE4ZWEtMzFiNjEwOTY0Mzkz',
 'YTkxOTBlZjItMzg2Mi00MTI4LWE5MzctOWQ4ZjI5YzFjNmZk',
 'NGZiNzU0YjMtZDFmMS00YzVmLWFiNGEtNWE0ZjI0MTdhYTQ1',
 'ODFkODVmMmYtZTAzMC00YjM3LWJlZTUtN2UzMmQzZGVlZmJm',
 'NzllNDNlNjYtMjU5NC00MTg4LWFjZGYtMzk3NTQ3N2Y3MThl',
 'YzNlYTBkZTQtNjA0Ny00NTZiLThiYmUtNGQ3Mjk4MjdhYmRm',
 'MmVhYzg5NDktYzQyZi00ZmUxLThiMjMtMmY0YjdhMjNmM2Rh',
 'NDY1ZjhmYTUtZjQwOC00MWNhLWIwOTgtNWUzNGYwYWJhOTE0',
 'NzA1ZDA5YTYtZDEyZS00YmFmLWI4YzEtYjc1NGI5ZGI0ZmUx',
 'MjQwYWQyMDctMzU0MC00NzA1LThjNjYtY2NjNWZlYmQx

In [30]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = AzureChatOpenAI(temperature=0.0,
    max_tokens=500,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name=chat_model,
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type    
)

### Let's try without using memory first! 

We can use RetrievalQA for that as it only focuses on "query" and "context" and does not require "chat_history". 

In [32]:
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})
retrieval_chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=retriever)

print("Chat with your docs!")
while True:
    query = input()
    if query in ["quit", "exit"]:
        break
    result = retrieval_chain({"query": query})
    print("User:")
    print(result['query'])
    print("AI:")
    print(result['result'])

Chat with your docs!
User:
What did the president say about Ketanji Brown Jackson?
AI:
The President nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and he described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
User:
Did he mention who she succeeded?
AI:
No, he did not mention who she succeeded.


In [34]:
from langchain.chains import ConversationalRetrievalChain

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=chat,
    retriever=retriever,
    memory=memory,
    chain_type="stuff",
    verbose=False
)

print("Chat with your docs!")
while True:
    query = input()
    result = qa_chain({"question": query})
    if query in ["quit", "exit"]:
        break
    print("User:")
    print(result["question"])
    print("AI:")
    print(result["answer"])
    

Chat with your docs!
User:
What did the president say about Ketanji Brown Jackson?
AI:
The President nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and he described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
User:
Did he mention who she succeeded?
AI:
The President mentioned Justice Stephen Breyer as the predecessor of Ketanji Brown Jackson.
